# Geocoding Resale Flat Addresses

In [1]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shapely 
from geopy.distance import geodesic
import geopandas
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree
import numpy as np
from geopy.distance import great_circle
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [2]:
# read in csv file of transactions 
df = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')

In [3]:
df2 = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')

In [4]:
# calculate remaining lease for df2 


# convert month column string to date time format 
df2['month'] = pd.to_datetime(df2['month'])
# extract year from 'month' column
df2['month'] = df2['month'].dt.year
# subtract lease duration used up from 99 years 
df2['remaining_lease'] = 99 - (df2['month']-df2['lease_commence_date'])

# check results 
df2

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2000,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,85
1,2000,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0,85
2,2000,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0,75
3,2000,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0,75
4,2000,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0,75
...,...,...,...,...,...,...,...,...,...,...,...
369646,2012,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0,72
369647,2012,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0,73
369648,2012,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0,75
369649,2012,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0,79


In [5]:
df['month'] = pd.to_datetime(df['month'])
# extract year from 'month' column
df['month'] = df['month'].dt.year
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0
...,...,...,...,...,...,...,...,...,...,...,...
37148,2016,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,82,488000.0
37149,2016,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,69,455000.0
37150,2016,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,74,778000.0
37151,2016,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,70,575000.0


In [6]:
# combine the two datasets 
new_df = pd.concat([df, df2], axis=0)
# check dataframe 
new_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0
...,...,...,...,...,...,...,...,...,...,...,...
369646,2012,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,72,476888.0
369647,2012,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,73,490000.0
369648,2012,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,75,488000.0
369649,2012,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,79,705000.0


In [7]:
# select only resale transactions in neighbourhoods identified as gentrified

# specify array of gentrifiedneighbourhoods 
array=['ANG MO KIO','CLEMENTI','QUEENSTOWN','BUKIT MERAH','GEYLANG','KALLANG','TOA PAYOH','BUKIT TIMAH','MARINE PARADE','TANGLIN','NOVENA']
new_df = new_df.loc[new_df['town'].isin(array)]


In [8]:
new_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0
...,...,...,...,...,...,...,...,...,...,...,...
369408,2012,TOA PAYOH,5 ROOM,10,LOR 7 TOA PAYOH,22 TO 24,120.0,Improved,1998,85,700000.0
369409,2012,TOA PAYOH,5 ROOM,79C,TOA PAYOH CTRL,37 TO 39,110.0,Improved,2009,96,894000.0
369410,2012,TOA PAYOH,EXECUTIVE,99A,LOR 2 TOA PAYOH,16 TO 18,145.0,Apartment,1993,80,860100.0
369411,2012,TOA PAYOH,EXECUTIVE,133,POTONG PASIR AVE 1,01 TO 03,149.0,Maisonette,1984,71,704000.0


In [9]:
# Let's combine the block and street name to form the address of our transacted unit.
new_df['address'] = new_df['block'] + " " + new_df['street_name']
new_df

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4
1,2015,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,541 ANG MO KIO AVE 10
2,2015,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,163 ANG MO KIO AVE 4
3,2015,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,446 ANG MO KIO AVE 10
4,2015,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,557 ANG MO KIO AVE 10
...,...,...,...,...,...,...,...,...,...,...,...,...
369408,2012,TOA PAYOH,5 ROOM,10,LOR 7 TOA PAYOH,22 TO 24,120.0,Improved,1998,85,700000.0,10 LOR 7 TOA PAYOH
369409,2012,TOA PAYOH,5 ROOM,79C,TOA PAYOH CTRL,37 TO 39,110.0,Improved,2009,96,894000.0,79C TOA PAYOH CTRL
369410,2012,TOA PAYOH,EXECUTIVE,99A,LOR 2 TOA PAYOH,16 TO 18,145.0,Apartment,1993,80,860100.0,99A LOR 2 TOA PAYOH
369411,2012,TOA PAYOH,EXECUTIVE,133,POTONG PASIR AVE 1,01 TO 03,149.0,Maisonette,1984,71,704000.0,133 POTONG PASIR AVE 1


In [10]:
# Obtain only unique addresses by removing duplicates 
address_list = new_df.drop_duplicates(subset='address', keep='first')
len(address_list)

1754

In [11]:
address_list = address_list['address'].tolist()

In [12]:
address_list

['174 ANG MO KIO AVE 4',
 '541 ANG MO KIO AVE 10',
 '163 ANG MO KIO AVE 4',
 '446 ANG MO KIO AVE 10',
 '557 ANG MO KIO AVE 10',
 '603 ANG MO KIO AVE 5',
 '709 ANG MO KIO AVE 8',
 '333 ANG MO KIO AVE 1',
 '109 ANG MO KIO AVE 4',
 '564 ANG MO KIO AVE 3',
 '218 ANG MO KIO AVE 1',
 '556 ANG MO KIO AVE 10',
 '156 ANG MO KIO AVE 4',
 '471 ANG MO KIO AVE 10',
 '434 ANG MO KIO AVE 10',
 '560 ANG MO KIO AVE 10',
 '332 ANG MO KIO AVE 1',
 '421 ANG MO KIO AVE 10',
 '506 ANG MO KIO AVE 8',
 '631 ANG MO KIO AVE 4',
 '153 ANG MO KIO AVE 5',
 '442 ANG MO KIO AVE 10',
 '558 ANG MO KIO AVE 10',
 '212 ANG MO KIO AVE 3',
 '152 ANG MO KIO AVE 5',
 '331 ANG MO KIO AVE 1',
 '121 ANG MO KIO AVE 3',
 '130 ANG MO KIO AVE 3',
 '646 ANG MO KIO AVE 6',
 '424 ANG MO KIO AVE 3',
 '584 ANG MO KIO AVE 3',
 '412 ANG MO KIO AVE 10',
 '612 ANG MO KIO AVE 4',
 '562 ANG MO KIO AVE 3',
 '223 ANG MO KIO AVE 1',
 '150 ANG MO KIO AVE 5',
 '468 ANG MO KIO AVE 10',
 '177 ANG MO KIO AVE 4',
 '415 ANG MO KIO AVE 10',
 '254 ANG MO

In [13]:
# Script obtained from https://gist.github.com/yuan-yexi/97a199feaaea61ba3eb5c2bd10279a03#file-hdb_resale_api_to_dataframe-py
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []
count = 0
for row in range(len(address_list)):
    #formulate query string  
    query_address = address_list[row]
    query_string='https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)
#Convert JSON into Python Object 
    data_geo_location=json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        blk_no.append(data_geo_location['results'][0]['BLK_NO'])
        road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        address.append(query_address)
        print (str(query_address) + " ,Lat: " + data_geo_location['results'][0]['LATITUDE'] + " Long: " + data_geo_location['results'][0]['LONGITUDE'])
    else:
        print ("No Results")

174 ANG MO KIO AVE 4 ,Lat: 1.375097469 Long: 103.83761899999999
541 ANG MO KIO AVE 10 ,Lat: 1.3739223919999999 Long: 103.8556214
163 ANG MO KIO AVE 4 ,Lat: 1.373548539 Long: 103.8381765
446 ANG MO KIO AVE 10 ,Lat: 1.367760951 Long: 103.8553571
557 ANG MO KIO AVE 10 ,Lat: 1.3716257019999998 Long: 103.8577361
603 ANG MO KIO AVE 5 ,Lat: 1.38020079 Long: 103.8357557
709 ANG MO KIO AVE 8 ,Lat: 1.3711372080000002 Long: 103.8476623
333 ANG MO KIO AVE 1 ,Lat: 1.3613425559999999 Long: 103.85169859999999
109 ANG MO KIO AVE 4 ,Lat: 1.370096538 Long: 103.8376878
564 ANG MO KIO AVE 3 ,Lat: 1.369848376 Long: 103.85940409999999
218 ANG MO KIO AVE 1 ,Lat: 1.365119086 Long: 103.84174250000001
556 ANG MO KIO AVE 10 ,Lat: 1.3720323630000002 Long: 103.8576254
156 ANG MO KIO AVE 4 ,Lat: 1.375495196 Long: 103.8399475
471 ANG MO KIO AVE 10 ,Lat: 1.363466006 Long: 103.85670290000002
434 ANG MO KIO AVE 10 ,Lat: 1.367891507 Long: 103.85345500000001
560 ANG MO KIO AVE 10 ,Lat: 1.3708162680000002 Long: 103.859192

44 SIMS DR ,Lat: 1.316235211 Long: 103.8768812
352 UBI AVE 1 ,Lat: 1.3252270179999999 Long: 103.8994002
11 EUNOS CRES ,Lat: 1.321734557 Long: 103.90379420000001
11 HAIG RD ,Lat: 1.31483627 Long: 103.89663730000001
67 CIRCUIT RD ,Lat: 1.3258485759999998 Long: 103.8879265
28 CASSIA CRES ,Lat: 1.30894453 Long: 103.8835301
125 ALJUNIED RD ,Lat: 1.323270062 Long: 103.8817522
123 PAYA LEBAR WAY ,Lat: 1.322756054 Long: 103.8832352
6 HAIG RD ,Lat: 1.313176188 Long: 103.89721490000001
5 PINE CL ,Lat: 1.307043059 Long: 103.8826884
45 JLN TIGA ,Lat: 1.308637777 Long: 103.8846448
8 MARINE TER ,Lat: 1.304364656 Long: 103.91610859999999
79 MARINE DR ,Lat: 1.301533134 Long: 103.90781199999999
29 MARINE CRES ,Lat: 1.302849836 Long: 103.91319720000001
166 STIRLING RD ,Lat: 1.29069966 Long: 103.8023629
83 C'WEALTH CL ,Lat: 1.3046786609999999 Long: 103.80042359999999
97 C'WEALTH CRES ,Lat: 1.306160472 Long: 103.8014957
95 C'WEALTH DR ,Lat: 1.305387463 Long: 103.8010632
98 C'WEALTH CRES ,Lat: 1.306822191 

26B JLN MEMBINA ,Lat: 1.283092071 Long: 103.8255924
76A REDHILL RD ,Lat: 1.28871632 Long: 103.8168546
110 SPOTTISWOODE PK RD ,Lat: 1.273638234 Long: 103.8364203
25 TELOK BLANGAH CRES ,Lat: 1.276336898 Long: 103.8191944
119C KIM TIAN RD ,Lat: 1.281489151 Long: 103.82767199999999
111B DEPOT RD ,Lat: 1.281165101 Long: 103.80847179999999
131B KIM TIAN RD ,Lat: 1.280917569 Long: 103.8283703
73A REDHILL RD ,Lat: 1.28793757 Long: 103.81679090000002
6 TOH YI DR ,Lat: 1.337773028 Long: 103.7751307
3 TOH YI DR ,Lat: 1.338774304 Long: 103.7752542
706 CLEMENTI WEST ST 2 ,Lat: 1.30676076 Long: 103.76236309999999
713 CLEMENTI WEST ST 2 ,Lat: 1.304127842 Long: 103.7622063
362 CLEMENTI AVE 2 ,Lat: 1.312716928 Long: 103.7699026
612 CLEMENTI WEST ST 1 ,Lat: 1.303442199 Long: 103.7686334
309 CLEMENTI AVE 4 ,Lat: 1.320919814 Long: 103.7663776
428 CLEMENTI AVE 3 ,Lat: 1.312511991 Long: 103.76326879999999
303 CLEMENTI AVE 4 ,Lat: 1.321329477 Long: 103.76481590000002
509 WEST COAST DR ,Lat: 1.311157758 Long:

43 TELOK BLANGAH RISE ,Lat: 1.270379512 Long: 103.823236
2 JLN BT MERAH ,Lat: 1.2866292129999999 Long: 103.8068619
44 TELOK BLANGAH DR ,Lat: 1.271408833 Long: 103.8108875
22 TELOK BLANGAH CRES ,Lat: 1.2765592159999999 Long: 103.81982190000001
28 HOY FATT RD ,Lat: 1.2867565440000002 Long: 103.81012170000001
64 TELOK BLANGAH DR ,Lat: 1.274506393 Long: 103.8105355
113 JLN BT MERAH ,Lat: 1.280277649 Long: 103.82572359999999
61 TELOK BLANGAH HTS ,Lat: 1.2757050079999999 Long: 103.8120071
86 REDHILL CL ,Lat: 1.286920062 Long: 103.8186736
22 HAVELOCK RD ,Lat: 1.2887848240000002 Long: 103.82826850000001
75 TELOK BLANGAH DR ,Lat: 1.274896709 Long: 103.8086953
50 MOH GUAN TER ,Lat: 1.28407985 Long: 103.83052550000001
146 JLN BT MERAH ,Lat: 1.277848766 Long: 103.8308115
69 TELOK BLANGAH HTS ,Lat: 1.275583428 Long: 103.810314
111A DEPOT RD ,Lat: 1.281122693 Long: 103.80868790000001
62 TELOK BLANGAH HTS ,Lat: 1.2752454309999999 Long: 103.81110670000001
39 JLN RUMAH TINGGI ,Lat: 1.2886333429999999 L

231 ANG MO KIO AVE 3 ,Lat: 1.368561036 Long: 103.8375252
472 ANG MO KIO AVE 10 ,Lat: 1.3631439269999999 Long: 103.8581186
502 ANG MO KIO AVE 5 ,Lat: 1.375724748 Long: 103.8499343
546 ANG MO KIO AVE 10 ,Lat: 1.373651239 Long: 103.8570222
522 ANG MO KIO AVE 5 ,Lat: 1.373115919 Long: 103.85194940000001
635 ANG MO KIO AVE 6 ,Lat: 1.380768961 Long: 103.84237840000002
596D ANG MO KIO ST 52 ,Lat: 1.3718713530000002 Long: 103.8510304
111 ANG MO KIO AVE 4 ,Lat: 1.37085149 Long: 103.8372982
101 ANG MO KIO AVE 3 ,Lat: 1.370261584 Long: 103.8394679
401 ANG MO KIO AVE 10 ,Lat: 1.361017656 Long: 103.85538759999999
714 ANG MO KIO AVE 6 ,Lat: 1.3710964980000002 Long: 103.84623149999999
596C ANG MO KIO ST 52 ,Lat: 1.371935719 Long: 103.8501433
45 TELOK BLANGAH DR ,Lat: 1.271757071 Long: 103.80956040000001
40 BEO CRES ,Lat: 1.289057892 Long: 103.827513
136 JLN BT MERAH ,Lat: 1.278287639 Long: 103.8275372
65 TELOK BLANGAH DR ,Lat: 1.274062304 Long: 103.81042409999999
46 JLN BT HO SWEE ,Lat: 1.288295797 L

...

29 BALAM RD ,Lat: 1.32999792 Long: 103.887048
9 PINE CL ,Lat: 1.309469497 Long: 103.8824375
2B GEYLANG SERAI ,Lat: 1.317405112 Long: 103.89777459999999
2D GEYLANG SERAI ,Lat: 1.317571243 Long: 103.89864770000001
120 PAYA LEBAR WAY ,Lat: 1.322594152 Long: 103.88429609999999
15 EUNOS CRES ,Lat: 1.32385971 Long: 103.90336359999999
65 MARINE DR ,Lat: 1.303405476 Long: 103.9097344
4 GHIM MOH RD ,Lat: 1.313193696 Long: 103.7881922
20 DOVER CRES ,Lat: 1.3067225740000001 Long: 103.7835241
62B STRATHMORE AVE ,Lat: 1.2944605759999999 Long: 103.810979
15 GHIM MOH RD ,Lat: 1.309143843 Long: 103.78841990000001
2 HOLLAND AVE ,Lat: 1.309344983 Long: 103.7955594
3 HOLLAND CL ,Lat: 1.308208713 Long: 103.7966389
12 DOVER CL EAST ,Lat: 1.304992003 Long: 103.78580749999999
28D DOVER CRES ,Lat: 1.306083152 Long: 103.7824724
92 LOR 4 TOA PAYOH ,Lat: 1.3380708159999999 Long: 103.8494572
235 LOR 8 TOA PAYOH ,Lat: 1.3389496920000001 Long: 103.85713890000001
83 LOR 2 TOA PAYOH ,Lat: 1.335787912 Long: 103.847992

27 DOVER CRES ,Lat: 1.305422783 Long: 103.7813204
28 GHIM MOH LINK ,Lat: 1.30887985 Long: 103.78480290000002
3 UPP ALJUNIED LANE ,Lat: 1.333992975 Long: 103.8798927
107 LOR 1 TOA PAYOH ,Lat: 1.341233588 Long: 103.84729399999999
58 LOR 4 TOA PAYOH ,Lat: 1.336746255 Long: 103.85052459999999
142 POTONG PASIR AVE 3 ,Lat: 1.333204067 Long: 103.8669926
227 LOR 8 TOA PAYOH ,Lat: 1.3395151440000002 Long: 103.8579708
14B LOR 7 TOA PAYOH ,Lat: 1.335738874 Long: 103.8584084
138 POTONG PASIR AVE 3 ,Lat: 1.33356221 Long: 103.86545859999998
86 LOR 2 TOA PAYOH ,Lat: 1.335762248 Long: 103.8476311
193 LOR 4 TOA PAYOH ,Lat: 1.331590056 Long: 103.85129509999999
155 LOR 1 TOA PAYOH ,Lat: 1.3327963109999998 Long: 103.8455886
218 LOR 8 TOA PAYOH ,Lat: 1.342725835 Long: 103.8551516
19 JOO SENG RD ,Lat: 1.334646362 Long: 103.8809946
44 LOR 5 TOA PAYOH ,Lat: 1.336231496 Long: 103.8540431
533 ANG MO KIO AVE 5 ,Lat: 1.3741692369999998 Long: 103.85300190000001
629 ANG MO KIO AVE 4 ,Lat: 1.38010475 Long: 103.84040

123 GEYLANG EAST CTRL ,Lat: 1.3181773509999999 Long: 103.8854258
3 MARINE TER ,Lat: 1.304106067 Long: 103.9177334
94 C'WEALTH DR ,Lat: 1.305586163 Long: 103.8007436
5 HOLLAND CL ,Lat: 1.307674422 Long: 103.79627159999998
150 MEI LING ST ,Lat: 1.294903079 Long: 103.8036993
52 LOR 6 TOA PAYOH ,Lat: 1.3382491429999999 Long: 103.852482
183 TOA PAYOH CTRL ,Lat: 1.333355962 Long: 103.8488657
8 JOO SENG RD ,Lat: 1.335201434 Long: 103.878703
7 LOR 7 TOA PAYOH ,Lat: 1.338053697 Long: 103.8572869
727 ANG MO KIO AVE 6 ,Lat: 1.3726551630000001 Long: 103.8460633
126 ANG MO KIO AVE 3 ,Lat: 1.3716499 Long: 103.8436442
8 TELOK BLANGAH CRES ,Lat: 1.277795391 Long: 103.81934129999999
30 TELOK BLANGAH RISE ,Lat: 1.272985283 Long: 103.820252
101 HENDERSON CRES ,Lat: 1.2907179979999999 Long: 103.8216252
121A KIM TIAN PL ,Lat: 1.282650681 Long: 103.8281785
58 TELOK BLANGAH HTS ,Lat: 1.274922927 Long: 103.81340809999999
109 CLEMENTI ST 11 ,Lat: 1.321980677 Long: 103.77077609999999
335 CLEMENTI AVE 2 ,Lat: 1.

233 LOR 8 TOA PAYOH ,Lat: 1.338283659 Long: 103.858468
84B LOR 2 TOA PAYOH ,Lat: 1.335765023 Long: 103.84680259999999
554 ANG MO KIO AVE 10 ,Lat: 1.3728200080000001 Long: 103.857784
636 ANG MO KIO AVE 6 ,Lat: 1.381152654 Long: 103.8433103
716 ANG MO KIO AVE 6 ,Lat: 1.371477831 Long: 103.84586879999999
47 MOH GUAN TER ,Lat: 1.2842058170000001 Long: 103.8310634
40 JLN RUMAH TINGGI ,Lat: 1.288632893 Long: 103.8105346
3 EVERTON PK ,Lat: 1.2772173009999999 Long: 103.838704
420 CLEMENTI AVE 1 ,Lat: 1.31004549 Long: 103.7691238
302 UBI AVE 1 ,Lat: 1.33002587 Long: 103.9009954
337 UBI AVE 1 ,Lat: 1.32598214 Long: 103.9015132
43A SIMS DR ,Lat: 1.3168188440000002 Long: 103.87663959999999
5 HAIG RD ,Lat: 1.312589174 Long: 103.8965912
99 C'WEALTH CRES ,Lat: 1.307434129 Long: 103.80114379999999
46 HOLLAND DR ,Lat: 1.308789494 Long: 103.79269559999999
80 STRATHMORE AVE ,Lat: 1.294561391 Long: 103.80969209999999
19 QUEEN'S CL ,Lat: 1.292382728 Long: 103.8001963
106 POTONG PASIR AVE 1 ,Lat: 1.33410033

312 CLEMENTI AVE 4 ,Lat: 1.319064417 Long: 103.7639598
322 CLEMENTI AVE 5 ,Lat: 1.3161529859999999 Long: 103.7663283
7 HAIG RD ,Lat: 1.313806612 Long: 103.8969105
2 MARINE TER ,Lat: 1.304467615 Long: 103.91817409999999
32 TANGLIN HALT RD ,Lat: 1.299620525 Long: 103.7993864
58 C'WEALTH DR ,Lat: 1.297769693 Long: 103.79709009999999
28 TANGLIN HALT RD ,Lat: 1.298543257 Long: 103.7998979
27 TANGLIN HALT RD ,Lat: 1.298324737 Long: 103.7998392
145 MEI LING ST ,Lat: 1.293813729 Long: 103.80502790000001
36 DOVER RD ,Lat: 1.299991656 Long: 103.78274300000001
185 TOA PAYOH CTRL ,Lat: 1.332727959 Long: 103.8499942
109 POTONG PASIR AVE 1 ,Lat: 1.334863952 Long: 103.8674116
216 LOR 8 TOA PAYOH ,Lat: 1.34247206 Long: 103.8541787
132 POTONG PASIR AVE 1 ,Lat: 1.3341754829999999 Long: 103.8656107
467 ANG MO KIO AVE 10 ,Lat: 1.365391066 Long: 103.85680359999999
No Results
96 HENDERSON RD ,Lat: 1.287542915 Long: 103.8213372
No Results
4 DELTA AVE ,Lat: 1.291828624 Long: 103.8265458
103 SPOTTISWOODE PK RD

In [14]:
# save coordinates to dataframe 
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

1706

In [15]:
# join the two dataframes based on the address column - only interested in latitude and longitude column of df_coordinates
final_df = new_df.merge(df_coordinates[["latitude", "longitude","address"]], on="address", how="outer").drop_duplicates().dropna()
# check dataframe 
final_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,1.375097469,103.83761899999999
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4,1.375097469,103.83761899999999
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4,1.375097469,103.83761899999999
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4,1.375097469,103.83761899999999
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4,1.375097469,103.83761899999999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89317,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,240000.0,16 MARINE TER,1.30437185,103.9140417
89318,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,233000.0,16 MARINE TER,1.30437185,103.9140417
89319,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,718000.0,125 POTONG PASIR AVE 1,1.33541494,103.86516879999999
89320,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,355000.0,727 CLEMENTI WEST ST 2,1.304624877,103.7638177


In [16]:
# convert lat long columns to floats 
final_df["latitude"] = pd.to_numeric(final_df["latitude"], downcast="float")
final_df["longitude"] = pd.to_numeric(final_df["longitude"], downcast="float")

# creating geodataframe by creating geometry column containing coordinates of flats 
gdf_resale = geopandas.GeoDataFrame(
    final_df, geometry=geopandas.points_from_xy(final_df['longitude'], final_df['latitude']))

gdf_resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,geometry
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89317,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,240000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437)
89318,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,233000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437)
89319,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,718000.0,125 POTONG PASIR AVE 1,1.335415,103.865166,POINT (103.86517 1.33541)
89320,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,355000.0,727 CLEMENTI WEST ST 2,1.304625,103.763817,POINT (103.76382 1.30462)


In [17]:
# setting crs of geometry to WGS84
gdf_resale.crs = {'init' :'epsg:4326'}

//anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [18]:
gdf_resale['town'].value_counts()

ANG MO KIO       21153
BUKIT MERAH      14154
TOA PAYOH        12990
QUEENSTOWN       11944
GEYLANG          11445
CLEMENTI         10825
MARINE PARADE     3155
BUKIT TIMAH       1014
Name: town, dtype: int64

In [19]:
gdf_resale['flat_type'].value_counts()

3 ROOM       48693
4 ROOM       21945
5 ROOM       11406
2 ROOM        2810
EXECUTIVE     1572
1 ROOM         254
Name: flat_type, dtype: int64

In [20]:
gdf_resale['flat_model'].value_counts()

New Generation        30710
Improved              27631
Standard              13594
Model A               11142
Simplified             1332
Maisonette              954
Adjoined flat           503
Apartment               474
Model A-Maisonette      250
Terrace                  75
DBSS                     14
Premium Apartment         1
Name: flat_model, dtype: int64

# Feature Engineering - Finding Nearest Neighbours and Shortest Distances

Code modified from https://automating-gis-processes.github.io/site/notebooks/L3/nearest-neighbor-faster.html

A faster and more memory efficient for conducting nearest neighbour analysis is based on BallTree from scikit-learn library, which includes the ability to calculate the distance between neighbors with various different distance metrics, e.g. euclidean and haversine distances. 

### Distance to Nearest Conservation Site 

In [21]:
# read in conservation site shapefile 
conservation_sites = gpd.read_file("conservation-area-map-shp/URA_CONSERVATION_AREA.shp")
conservation_sites 

,OBJECTID,GROUP_NAME,DESCRIPTIO,DATE_OF_GA,PHOTO_LINK,MORE_INFO,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,Former Royal Air Force (RAF) Seletar,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,43D4A11D89151BB0,2015-11-05,32414.3012,43715.8915,602.945195,23626.590132,"POLYGON ((32507.656 43774.848, 32502.216 43760..."
1,2,Former Royal Air Force (RAF) Seletar,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,ECE3A4115D6CC8E1,2015-11-05,32464.4286,43880.4604,438.898190,12297.463374,"POLYGON ((32543.426 43880.956, 32538.575 43861..."
2,3,Malabar Mosque,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,BBD6C6EA373D2F8E,2015-11-05,30950.5660,31943.0939,101.806547,650.077537,"POLYGON ((30961.880 31931.908, 30951.824 31924..."
3,4,Leong San See Temple,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,04595AD3564C60F4,2015-11-05,30603.5000,33063.7951,186.312295,1756.716746,"POLYGON ((30638.589 33053.239, 30620.404 33035..."
4,5,Former Chee Kong Tong (Thekchen Choling) Entra...,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,388E45258ED2A225,2015-11-05,31014.6595,32707.2165,91.130775,458.594306,"POLYGON ((30999.548 32715.301, 31007.745 32724..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,244,White House Park/Nassim Rd(7) Lot 61-22 TS 25 ...,None,29 November 1991,None,http://www.ura.gov.sg/conservationportal/consm...,5AFCAE815E84E699,2014-01-30,26848.2654,32495.2566,205.663318,2577.190523,"POLYGON ((26844.444 32458.672, 26815.709 32483..."
250,245,White House Park/Nassim Rd(2) Lot 464 TS 25 (2...,None,29 November 1991,None,http://www.ura.gov.sg/conservationportal/consm...,1459F3FF392FF6C3,2014-01-30,26621.7449,33129.6617,301.175699,5438.518655,"POLYGON ((26605.959 33103.541, 26565.102 33121..."
251,246,Block 1 of Former Beach Road Camp,None,9 October 2002,None,http://www.ura.gov.sg/conservationportal/consm...,1AE17FD6BF304ABD,2014-01-30,30544.0244,30906.4672,148.557506,940.602066,"POLYGON ((30522.515 30885.765, 30551.705 30935..."
252,247,Fort Canning and Coleman Street,None,21 November 2005,http://farm8.staticflickr.com/7109/7028884905_...,http://www.ura.gov.sg/conservationportal/consm...,6BB9026C969F5F65,2014-01-30,29506.3625,30705.8290,2142.170103,252339.562061,"POLYGON ((29351.449 30614.351, 29350.669 30619..."


In [22]:
# creating a geometry column for point location 
gdf = geopandas.GeoDataFrame(
    conservation_sites, geometry=geopandas.points_from_xy(conservation_sites.X_ADDR, conservation_sites.Y_ADDR))
gdf.crs = {'init' :'epsg:3414'}
gdf

//anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,OBJECTID,GROUP_NAME,DESCRIPTIO,DATE_OF_GA,PHOTO_LINK,MORE_INFO,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,Former Royal Air Force (RAF) Seletar,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,43D4A11D89151BB0,2015-11-05,32414.3012,43715.8915,602.945195,23626.590132,POINT (32414.301 43715.891)
1,2,Former Royal Air Force (RAF) Seletar,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,ECE3A4115D6CC8E1,2015-11-05,32464.4286,43880.4604,438.898190,12297.463374,POINT (32464.429 43880.460)
2,3,Malabar Mosque,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,BBD6C6EA373D2F8E,2015-11-05,30950.5660,31943.0939,101.806547,650.077537,POINT (30950.566 31943.094)
3,4,Leong San See Temple,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,04595AD3564C60F4,2015-11-05,30603.5000,33063.7951,186.312295,1756.716746,POINT (30603.500 33063.795)
4,5,Former Chee Kong Tong (Thekchen Choling) Entra...,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,388E45258ED2A225,2015-11-05,31014.6595,32707.2165,91.130775,458.594306,POINT (31014.660 32707.216)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,244,White House Park/Nassim Rd(7) Lot 61-22 TS 25 ...,None,29 November 1991,None,http://www.ura.gov.sg/conservationportal/consm...,5AFCAE815E84E699,2014-01-30,26848.2654,32495.2566,205.663318,2577.190523,POINT (26848.265 32495.257)
250,245,White House Park/Nassim Rd(2) Lot 464 TS 25 (2...,None,29 November 1991,None,http://www.ura.gov.sg/conservationportal/consm...,1459F3FF392FF6C3,2014-01-30,26621.7449,33129.6617,301.175699,5438.518655,POINT (26621.745 33129.662)
251,246,Block 1 of Former Beach Road Camp,None,9 October 2002,None,http://www.ura.gov.sg/conservationportal/consm...,1AE17FD6BF304ABD,2014-01-30,30544.0244,30906.4672,148.557506,940.602066,POINT (30544.024 30906.467)
252,247,Fort Canning and Coleman Street,None,21 November 2005,http://farm8.staticflickr.com/7109/7028884905_...,http://www.ura.gov.sg/conservationportal/consm...,6BB9026C969F5F65,2014-01-30,29506.3625,30705.8290,2142.170103,252339.562061,POINT (29506.362 30705.829)


In [23]:
# reproject data into WGS84 

gdf=gdf.to_crs({'init' :'epsg:4326'})

# check reproject has been executed properly 
gdf

//anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,OBJECTID,GROUP_NAME,DESCRIPTIO,DATE_OF_GA,PHOTO_LINK,MORE_INFO,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,Former Royal Air Force (RAF) Seletar,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,43D4A11D89151BB0,2015-11-05,32414.3012,43715.8915,602.945195,23626.590132,POINT (103.87298 1.41163)
1,2,Former Royal Air Force (RAF) Seletar,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,ECE3A4115D6CC8E1,2015-11-05,32464.4286,43880.4604,438.898190,12297.463374,POINT (103.87344 1.41311)
2,3,Malabar Mosque,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,BBD6C6EA373D2F8E,2015-11-05,30950.5660,31943.0939,101.806547,650.077537,POINT (103.85983 1.30516)
3,4,Leong San See Temple,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,04595AD3564C60F4,2015-11-05,30603.5000,33063.7951,186.312295,1756.716746,POINT (103.85671 1.31529)
4,5,Former Chee Kong Tong (Thekchen Choling) Entra...,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,388E45258ED2A225,2015-11-05,31014.6595,32707.2165,91.130775,458.594306,POINT (103.86041 1.31207)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,244,White House Park/Nassim Rd(7) Lot 61-22 TS 25 ...,None,29 November 1991,None,http://www.ura.gov.sg/conservationportal/consm...,5AFCAE815E84E699,2014-01-30,26848.2654,32495.2566,205.663318,2577.190523,POINT (103.82297 1.31015)
250,245,White House Park/Nassim Rd(2) Lot 464 TS 25 (2...,None,29 November 1991,None,http://www.ura.gov.sg/conservationportal/consm...,1459F3FF392FF6C3,2014-01-30,26621.7449,33129.6617,301.175699,5438.518655,POINT (103.82093 1.31589)
251,246,Block 1 of Former Beach Road Camp,None,9 October 2002,None,http://www.ura.gov.sg/conservationportal/consm...,1AE17FD6BF304ABD,2014-01-30,30544.0244,30906.4672,148.557506,940.602066,POINT (103.85618 1.29578)
252,247,Fort Canning and Coleman Street,None,21 November 2005,http://farm8.staticflickr.com/7109/7028884905_...,http://www.ura.gov.sg/conservationportal/consm...,6BB9026C969F5F65,2014-01-30,29506.3625,30705.8290,2142.170103,252339.562061,POINT (103.84685 1.29397)


In [25]:
# Define function for nearest neighbour search 
# Script modified from https://automating-gis-processes.github.io/site/notebooks/L3/nearest-neighbor-faster.html

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)


def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name

    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]

    # Ensure that the index corresponds the one in left_gdf
    closest_points = closest_points.reset_index(drop=True)

    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius

    return closest_points

Nearest Neighbour Search 

In [26]:
# Find closest conservation site for each flat and distance based on haversine distance
# Note: haversine distance which is implemented here is a bit slower than using e.g. 'euclidean' metric
# but useful for deriving distance between points in meters
closest_sites = nearest_neighbor(gdf_resale, gdf, return_dist=True)

# Check result
closest_sites

,OBJECTID,GROUP_NAME,DESCRIPTIO,DATE_OF_GA,PHOTO_LINK,MORE_INFO,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry,distance
0,35,Kiew Lee Tong Temple,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,E3DD00DCCF1F7E11,2015-11-05,27475.7811,38572.4348,221.382246,2529.883155,POINT (103.82861 1.36511),1036.226326
1,35,Kiew Lee Tong Temple,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,E3DD00DCCF1F7E11,2015-11-05,27475.7811,38572.4348,221.382246,2529.883155,POINT (103.82861 1.36511),1036.226326
2,35,Kiew Lee Tong Temple,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,E3DD00DCCF1F7E11,2015-11-05,27475.7811,38572.4348,221.382246,2529.883155,POINT (103.82861 1.36511),1036.226326
3,35,Kiew Lee Tong Temple,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,E3DD00DCCF1F7E11,2015-11-05,27475.7811,38572.4348,221.382246,2529.883155,POINT (103.82861 1.36511),1036.226326
4,35,Kiew Lee Tong Temple,None,6 June 2014,None,http://www.ura.gov.sg/conservationportal/consm...,E3DD00DCCF1F7E11,2015-11-05,27475.7811,38572.4348,221.382246,2529.883155,POINT (103.82861 1.36511),1036.226326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,204,Joo Chiat Conservation Area (Ph2) - 445 to 459...,None,1 December 2003,None,http://www.ura.gov.sg/conservationportal/consm...,AF9B9DDA79CA9732,2014-01-30,36824.2523,32402.0981,229.259445,2730.986047,POINT (103.91261 1.30931),206.692805
86676,204,Joo Chiat Conservation Area (Ph2) - 445 to 459...,None,1 December 2003,None,http://www.ura.gov.sg/conservationportal/consm...,AF9B9DDA79CA9732,2014-01-30,36824.2523,32402.0981,229.259445,2730.986047,POINT (103.91261 1.30931),206.692805
86677,104,St. Andrew's School,None,1 December 2003,http://farm9.staticflickr.com/8149/7176660231_...,http://www.ura.gov.sg/conservationportal/consm...,3E9C87A92DADFE9B,2014-01-30,31587.1535,34758.2321,1094.760673,66565.822010,POINT (103.86555 1.33062),134.881667
86678,251,Jalan Jurong Kechil,None,1 December 2003,http://farm8.staticflickr.com/7078/7370379400_...,http://www.ura.gov.sg/conservationportal/consm...,2EDB28F8E46345E0,2014-01-30,21289.3043,35859.3878,209.391099,2033.660234,POINT (103.77302 1.34057),1397.163127


In [27]:
# Check that there are the same number of closest sites as resale flats 
print(len(closest_sites), '==', len(gdf_resale))

86680 == 86680


In [28]:
gdf_resale=gdf_resale.reset_index(drop=True)
gdf_resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,geometry
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,240000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437)
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,233000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437)
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,718000.0,125 POTONG PASIR AVE 1,1.335415,103.865166,POINT (103.86517 1.33541)
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,355000.0,727 CLEMENTI WEST ST 2,1.304625,103.763817,POINT (103.76382 1.30462)


In [29]:
# combine the neighbouring datasets 

# Rename the geometry of closest sites gdf so that we can easily identify it
closest_sites = closest_sites.rename(columns={'geometry': 'closest_site_geom'})


In [30]:
# subset of original dataframe
closest_sites_sub = closest_sites[['closest_site_geom','distance']]
closest_sites_sub

,closest_site_geom,distance
0,POINT (103.82861 1.36511),1036.226326
1,POINT (103.82861 1.36511),1036.226326
2,POINT (103.82861 1.36511),1036.226326
3,POINT (103.82861 1.36511),1036.226326
4,POINT (103.82861 1.36511),1036.226326
...,...,...
86675,POINT (103.91261 1.30931),206.692805
86676,POINT (103.91261 1.30931),206.692805
86677,POINT (103.86555 1.33062),134.881667
86678,POINT (103.77302 1.34057),1397.163127


In [31]:
# Merge the datasets by index 
resale = gdf_resale.join(closest_sites_sub)
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,geometry,closest_site_geom,distance
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,240000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,233000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,718000.0,125 POTONG PASIR AVE 1,1.335415,103.865166,POINT (103.86517 1.33541),POINT (103.86555 1.33062),134.881667
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,355000.0,727 CLEMENTI WEST ST 2,1.304625,103.763817,POINT (103.76382 1.30462),POINT (103.77302 1.34057),1397.163127


### Distance to CBD

In [32]:
# Set Raffles Place MRT station as proxy for CBD 
# Returns distance in km 

def distance_to_mid(lat, lon):
    cbd = (1.2830, 103.8513) 
    flat = (lat, lon)
    return great_circle(cbd, flat).km

resale['cbd_distance'] = resale.apply(lambda x: distance_to_mid(x.latitude, x.longitude), axis=1)
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,geometry,closest_site_geom,distance,cbd_distance
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,240000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,233000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,718000.0,125 POTONG PASIR AVE 1,1.335415,103.865166,POINT (103.86517 1.33541),POINT (103.86555 1.33062),134.881667,6.028659
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,355000.0,727 CLEMENTI WEST ST 2,1.304625,103.763817,POINT (103.76382 1.30462),POINT (103.77302 1.34057),1397.163127,10.018078


### Distance to Nearest Public Transport Stops (MRT and Bus)

In [33]:
# Read in MRT location data 
mrt_stations = pd.read_csv('mrtsg.csv')
mrt_stations

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,153,BARTLEY MRT STATION,CC12,33168.3039,36108.7003,1.342828,103.879746,YELLOW
4,115,BAYFRONT MRT STATION,DT16,30867.0093,29368.6250,1.281874,103.859073,BLUE
...,...,...,...,...,...,...,...,...
139,175,WOODLANDS SOUTH MRT STATION,TE3,23607.8309,45444.7113,1.427260,103.793863,OTHERS
140,146,WOODLEIGH MRT STATION,NE11,32173.3186,35706.3794,1.339190,103.870808,PURPLE
141,6,YEW TEE MRT STATION,NS5,18438.9791,42158.0124,1.397535,103.747431,RED
142,41,YIO CHU KANG MRT STATION,NS15,29294.1283,40413.0820,1.381756,103.844944,RED


In [34]:
# convert lat long columns to floats 
gdf_mrt = geopandas.GeoDataFrame(
    mrt_stations, geometry=geopandas.points_from_xy(mrt_stations['Longitude'], mrt_stations['Latitude']))

gdf_mrt

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR,geometry
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED,POINT (103.80100 1.44059)
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN,POINT (103.88289 1.31643)
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED,POINT (103.84955 1.36993)
3,153,BARTLEY MRT STATION,CC12,33168.3039,36108.7003,1.342828,103.879746,YELLOW,POINT (103.87975 1.34283)
4,115,BAYFRONT MRT STATION,DT16,30867.0093,29368.6250,1.281874,103.859073,BLUE,POINT (103.85907 1.28187)
...,...,...,...,...,...,...,...,...,...
139,175,WOODLANDS SOUTH MRT STATION,TE3,23607.8309,45444.7113,1.427260,103.793863,OTHERS,POINT (103.79386 1.42726)
140,146,WOODLEIGH MRT STATION,NE11,32173.3186,35706.3794,1.339190,103.870808,PURPLE,POINT (103.87081 1.33919)
141,6,YEW TEE MRT STATION,NS5,18438.9791,42158.0124,1.397535,103.747431,RED,POINT (103.74743 1.39754)
142,41,YIO CHU KANG MRT STATION,NS15,29294.1283,40413.0820,1.381756,103.844944,RED,POINT (103.84494 1.38176)


In [35]:
# find closest mrt stations for each resale flat 
closest_mrt = nearest_neighbor(gdf_resale, gdf_mrt, return_dist=True)

# Check results
closest_mrt

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR,geometry,distance
0,160,MARYMOUNT MRT STATION,CC16,28679.3688,36758.7136,1.348707,103.839421,YELLOW,POINT (103.83942 1.34871),730.029000
1,160,MARYMOUNT MRT STATION,CC16,28679.3688,36758.7136,1.348707,103.839421,YELLOW,POINT (103.83942 1.34871),730.029000
2,160,MARYMOUNT MRT STATION,CC16,28679.3688,36758.7136,1.348707,103.839421,YELLOW,POINT (103.83942 1.34871),730.029000
3,160,MARYMOUNT MRT STATION,CC16,28679.3688,36758.7136,1.348707,103.839421,YELLOW,POINT (103.83942 1.34871),730.029000
4,160,MARYMOUNT MRT STATION,CC16,28679.3688,36758.7136,1.348707,103.839421,YELLOW,POINT (103.83942 1.34871),730.029000
...,...,...,...,...,...,...,...,...,...,...
86675,31,KEMBANGAN MRT STATION,EW6,36862.1237,33699.3568,1.321038,103.912928,GREEN,POINT (103.91293 1.32104),462.438213
86676,31,KEMBANGAN MRT STATION,EW6,36862.1237,33699.3568,1.321038,103.912928,GREEN,POINT (103.91293 1.32104),462.438213
86677,176,POTONG PASIR MRT STATION,NE10,31977.1677,34842.7289,1.331380,103.869046,PURPLE,POINT (103.86905 1.33138),444.669074
86678,15,CLEMENTI MRT STATION,EW23,20430.0600,33026.6075,1.314954,103.765317,GREEN,POINT (103.76532 1.31495),320.164404


In [36]:
# Rename the geometry of closest stops gdf so that we can easily identify it
closest_mrt = closest_mrt.rename(columns={'geometry': 'closest_mrt_geom'})
closest_mrt = closest_mrt.rename(columns={'distance': 'distance_mrt'})
closest_mrt = closest_mrt[['STN_NAME','closest_mrt_geom','distance_mrt']]
closest_mrt_sub = closest_mrt[['distance_mrt']]


In [37]:
# join to existing resale dataset 
resale = resale.join(closest_mrt_sub)

In [38]:
# Read in bus stops location data 
bus_stops = gpd.read_file('BusStopLocation_Jan2020/BusStop.shp')
bus_stops.crs = {'init' :'epsg:3414'}

//anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [39]:
bus_stops = bus_stops.to_crs({'init' :'epsg:4326'})

In [40]:
bus_stops

,BUS_STOP_N,BUS_ROOF_N,LOC_DESC,geometry
0,78221,B06,None,POINT (103.96117 1.37407)
1,63359,B01,HOUGANG SWIM CPLX,POINT (103.88782 1.36941)
2,64141,B13,AFT JLN TELAWI,POINT (103.90822 1.36469)
3,83139,B07,AFT JOO CHIAT PL,POINT (103.90997 1.31454)
4,55231,B02,OPP SBST EAST DISTRICT,POINT (103.84832 1.38563)
...,...,...,...,...
5035,65541,B01,BEF SUMANG LANE,POINT (103.89126 1.40351)
5036,67691,UNK,BLK 338A,POINT (103.88950 1.39891)
5037,67699,UNK,BLK 334A,POINT (103.88983 1.39872)
5038,67761,B02A,OPP BLK 326D,POINT (103.88754 1.39607)


In [41]:
# find closest bus stops for each resale flat 
closest_bus_stop = nearest_neighbor(gdf_resale, bus_stops, return_dist=True)
# Check results
closest_bus_stop


,BUS_STOP_N,BUS_ROOF_N,LOC_DESC,geometry,distance
0,54199,B04,BLK 163,POINT (103.83762 1.37422),23.365871
1,54199,B04,BLK 163,POINT (103.83762 1.37422),23.365871
2,54199,B04,BLK 163,POINT (103.83762 1.37422),23.365871
3,54199,B04,BLK 163,POINT (103.83762 1.37422),23.365871
4,54199,B04,BLK 163,POINT (103.83762 1.37422),23.365871
...,...,...,...,...,...
86675,92191,B11,OPP CP C3,POINT (103.91478 1.30247),96.899174
86676,92191,B11,OPP CP C3,POINT (103.91478 1.30247),96.899174
86677,61079,B06,BLK 121,POINT (103.86517 1.33600),15.509091
86678,17041,B07,BLK 726 CLEMENTI WEST MKT,POINT (103.76452 1.30424),78.858626


In [42]:
# Rename the geometry of closest stops gdf so that we can easily identify it
closest_bus_stop = closest_bus_stop.rename(columns={'geometry': 'closest_bus_stop_geom'})
closest_bus_stop = closest_bus_stop.rename(columns={'distance': 'distance_bus_stop'})
closest_bus_stop = closest_bus_stop[['distance_bus_stop']]


In [43]:
closest_bus_stop

,distance_bus_stop
0,23.365871
1,23.365871
2,23.365871
3,23.365871
4,23.365871
...,...
86675,96.899174
86676,96.899174
86677,15.509091
86678,78.858626


In [44]:
# join to existing resale dataset 
resale = resale.join(closest_bus_stop)


In [45]:
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address,latitude,longitude,geometry,closest_site_geom,distance,cbd_distance,distance_mrt,distance_bus_stop
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,275000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,310000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,253000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,290000.0,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,240000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,233000.0,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,718000.0,125 POTONG PASIR AVE 1,1.335415,103.865166,POINT (103.86517 1.33541),POINT (103.86555 1.33062),134.881667,6.028659,444.669074,15.509091
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,355000.0,727 CLEMENTI WEST ST 2,1.304625,103.763817,POINT (103.76382 1.30462),POINT (103.77302 1.34057),1397.163127,10.018078,320.164404,78.858626


### Distance to Nearest Shopping Mall

In [46]:
# read in location data 
shopping_malls = pd.read_csv('shopping malls.csv')
shopping_malls = shopping_malls[['latitude','longitude','address']]
shopping_malls

,latitude,longitude,address
0,1.362117,103.990204,Changi Airport Terminal 1
1,1.360332,103.989670,Jewel Changi Airport
2,1.303023,103.852876,Sim Lim Square
3,1.371936,103.845974,Broadway Plaza
4,1.392094,103.894980,Compass One
...,...,...,...
197,1.428114,103.836063,Northpoint City
198,1.302983,103.836284,Knightsbridge
199,1.334845,103.720462,Taman Jurong Shopping Centre
200,1.437131,103.795290,888 Plaza


In [47]:
# creating geometry column 
gdf_shopping_malls = geopandas.GeoDataFrame(
    shopping_malls, geometry=geopandas.points_from_xy(shopping_malls['longitude'], shopping_malls['latitude']))

gdf_shopping_malls

,latitude,longitude,address,geometry
0,1.362117,103.990204,Changi Airport Terminal 1,POINT (103.99020 1.36212)
1,1.360332,103.989670,Jewel Changi Airport,POINT (103.98967 1.36033)
2,1.303023,103.852876,Sim Lim Square,POINT (103.85288 1.30302)
3,1.371936,103.845974,Broadway Plaza,POINT (103.84597 1.37194)
4,1.392094,103.894980,Compass One,POINT (103.89498 1.39209)
...,...,...,...,...
197,1.428114,103.836063,Northpoint City,POINT (103.83606 1.42811)
198,1.302983,103.836284,Knightsbridge,POINT (103.83628 1.30298)
199,1.334845,103.720462,Taman Jurong Shopping Centre,POINT (103.72046 1.33484)
200,1.437131,103.795290,888 Plaza,POINT (103.79529 1.43713)


In [48]:
# find closest mall for each resale flat 
closest_mall = nearest_neighbor(gdf_resale, gdf_shopping_malls, return_dist=True)

# Check results
closest_mall

,latitude,longitude,address,geometry,distance
0,1.371936,103.845974,Broadway Plaza,POINT (103.84597 1.37194),933.145397
1,1.371936,103.845974,Broadway Plaza,POINT (103.84597 1.37194),933.145397
2,1.371936,103.845974,Broadway Plaza,POINT (103.84597 1.37194),933.145397
3,1.371936,103.845974,Broadway Plaza,POINT (103.84597 1.37194),933.145397
4,1.371936,103.845974,Broadway Plaza,POINT (103.84597 1.37194),933.145397
...,...,...,...,...,...
86675,1.301080,103.905282,Parkway Parade,POINT (103.90528 1.30108),977.791420
86676,1.301080,103.905282,Parkway Parade,POINT (103.90528 1.30108),977.791420
86677,1.324222,103.864549,R&F Mall,POINT (103.86455 1.32422),306.029520
86678,1.303743,103.766093,West Coast Plaza,POINT (103.76609 1.30374),254.161399


In [49]:
# Rename the geometry of closest stops gdf so that we can easily identify it
closest_mall = closest_mall.rename(columns={'geometry': 'closest_mall'})
closest_mall = closest_mall.rename(columns={'distance': 'distance_mall'})
closest_mall = closest_mall[['distance_mall']]
closest_mall

,distance_mall
0,933.145397
1,933.145397
2,933.145397
3,933.145397
4,933.145397
...,...
86675,977.791420
86676,977.791420
86677,306.029520
86678,254.161399


In [50]:
# join to existing resale dataset 
resale = resale.join(closest_mall)
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,address,latitude,longitude,geometry,closest_site_geom,distance,cbd_distance,distance_mrt,distance_bus_stop,distance_mall
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,...,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,...,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,...,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,...,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,...,174 ANG MO KIO AVE 4,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,...,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,...,16 MARINE TER,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,...,125 POTONG PASIR AVE 1,1.335415,103.865166,POINT (103.86517 1.33541),POINT (103.86555 1.33062),134.881667,6.028659,444.669074,15.509091,306.029520
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,...,727 CLEMENTI WEST ST 2,1.304625,103.763817,POINT (103.76382 1.30462),POINT (103.77302 1.34057),1397.163127,10.018078,320.164404,78.858626,254.161399


### Distance to Nearest Good School

In [51]:
# read in data of primary schools 
prestige_schools = pd.read_csv('primary schools.csv')
prestige_schools = prestige_schools[['school_name','address','latitude','longitude']]
prestige_schools

,school_name,address,latitude,longitude
0,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,1.433681,103.832924
1,AI TONG SCHOOL,100 Bright Hill Drive,1.360583,103.833020
2,ANGLO-CHINESE SCHOOL (PRIMARY),50 BARKER ROAD,1.318719,103.835345
3,CANBERRA PRIMARY SCHOOL,21 ADMIRALTY DRIVE,1.451064,103.815971
4,CATHOLIC HIGH SCHOOL,9 BISHAN STREET 22,1.354789,103.844934
5,CHIJ (KATONG) PRIMARY,17 MARTIA ROAD,1.306527,103.910922
6,CHIJ PRIMARY (TOA PAYOH),628 Lorong 1 Toa Payoh,1.332818,103.841918
7,CHONGFU SCHOOL,170 YISHUN AVENUE 6,1.438462,103.839071
8,CHONGZHENG PRIMARY SCHOOL,1 TAMPINES STREET 21,1.350595,103.951225
9,ELIAS PARK PRIMARY SCHOOL,11 PASIR RIS STREET 52,1.375057,103.945289


In [52]:
# creating geometry column 
gdf_pri_sch = geopandas.GeoDataFrame(
    prestige_schools, geometry=geopandas.points_from_xy(prestige_schools['longitude'], prestige_schools['latitude']))

gdf_pri_sch

,school_name,address,latitude,longitude,geometry
0,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,1.433681,103.832924,POINT (103.83292 1.43368)
1,AI TONG SCHOOL,100 Bright Hill Drive,1.360583,103.833020,POINT (103.83302 1.36058)
2,ANGLO-CHINESE SCHOOL (PRIMARY),50 BARKER ROAD,1.318719,103.835345,POINT (103.83535 1.31872)
3,CANBERRA PRIMARY SCHOOL,21 ADMIRALTY DRIVE,1.451064,103.815971,POINT (103.81597 1.45106)
4,CATHOLIC HIGH SCHOOL,9 BISHAN STREET 22,1.354789,103.844934,POINT (103.84493 1.35479)
5,CHIJ (KATONG) PRIMARY,17 MARTIA ROAD,1.306527,103.910922,POINT (103.91092 1.30653)
6,CHIJ PRIMARY (TOA PAYOH),628 Lorong 1 Toa Payoh,1.332818,103.841918,POINT (103.84192 1.33282)
7,CHONGFU SCHOOL,170 YISHUN AVENUE 6,1.438462,103.839071,POINT (103.83907 1.43846)
8,CHONGZHENG PRIMARY SCHOOL,1 TAMPINES STREET 21,1.350595,103.951225,POINT (103.95123 1.35059)
9,ELIAS PARK PRIMARY SCHOOL,11 PASIR RIS STREET 52,1.375057,103.945289,POINT (103.94529 1.37506)


In [53]:
# find closest mall for each resale flat 
closest_school = nearest_neighbor(gdf_resale, gdf_pri_sch, return_dist=True)

# Check results 
closest_school

,school_name,address,latitude,longitude,geometry,distance
0,MAYFLOWER PRIMARY SCHOOL,6 ANG MO KIO AVE 2,1.376068,103.835747,POINT (103.83575 1.37607),209.449588
1,MAYFLOWER PRIMARY SCHOOL,6 ANG MO KIO AVE 2,1.376068,103.835747,POINT (103.83575 1.37607),209.449588
2,MAYFLOWER PRIMARY SCHOOL,6 ANG MO KIO AVE 2,1.376068,103.835747,POINT (103.83575 1.37607),209.449588
3,MAYFLOWER PRIMARY SCHOOL,6 ANG MO KIO AVE 2,1.376068,103.835747,POINT (103.83575 1.37607),209.449588
4,MAYFLOWER PRIMARY SCHOOL,6 ANG MO KIO AVE 2,1.376068,103.835747,POINT (103.83575 1.37607),209.449588
...,...,...,...,...,...,...
86675,TAO NAN SCHOOL,49 MARINE CRESCENT,1.304632,103.911204,POINT (103.91120 1.30463),315.359851
86676,TAO NAN SCHOOL,49 MARINE CRESCENT,1.304632,103.911204,POINT (103.91120 1.30463),315.359851
86677,ST. ANDREW'S JUNIOR SCHOOL,2 FRANCIS THOMAS DRIVE,1.331389,103.865129,POINT (103.86513 1.33139),107.350259
86678,KEMING PRIMARY SCHOOL,90 BUKIT BATOK EAST AVENUE 6,1.345245,103.756265,POINT (103.75626 1.34525),1363.610294


In [54]:
# Rename the geometry of closest stops gdf so that we can easily identify it
closest_school = closest_school.rename(columns={'geometry': 'closest_school'})
closest_school = closest_school.rename(columns={'distance': 'distance_school'})
closest_school = closest_school[['distance_school']]
closest_school

,distance_school
0,209.449588
1,209.449588
2,209.449588
3,209.449588
4,209.449588
...,...
86675,315.359851
86676,315.359851
86677,107.350259
86678,1363.610294


In [55]:
# join to existing resale dataset 
resale = resale.join(closest_school)
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,latitude,longitude,geometry,closest_site_geom,distance,cbd_distance,distance_mrt,distance_bus_stop,distance_mall,distance_school
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,...,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,...,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,...,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,...,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,...,1.375098,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,...,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420,315.359851
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,...,1.304372,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420,315.359851
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,...,1.335415,103.865166,POINT (103.86517 1.33541),POINT (103.86555 1.33062),134.881667,6.028659,444.669074,15.509091,306.029520,107.350259
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,...,1.304625,103.763817,POINT (103.76382 1.30462),POINT (103.77302 1.34057),1397.163127,10.018078,320.164404,78.858626,254.161399,1363.610294


### Distance to Nearest Hawker Centre 

In [56]:
# read in location data 
hawker_centre = gpd.read_file('hawkercentre/HAWKERCENTRE.shp')
hawker_centre = hawker_centre[['ADDRESSSTR','geometry']]
# set crs 
hawker_centre.crs = {'init' :'epsg:3414'}

//anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [57]:
hawker_centre = hawker_centre.to_crs(epsg=4326)
hawker_centre

,ADDRESSSTR,geometry
0,Commonwealth Drive,POINT (103.79788 1.29971)
1,Onan Road,POINT (103.90183 1.30941)
2,Empress Road,POINT (103.80564 1.31625)
3,Eunos Crescent,POINT (103.90426 1.32033)
4,Geylang Bahru,POINT (103.87000 1.32146)
...,...,...
120,Lorong 5 Toa Payoh,POINT (103.85294 1.33604)
121,Lorong 7 Toa Payoh,POINT (103.85704 1.33523)
122,East Coast Parkway,POINT (103.93502 1.30668)
123,New Upper Changi Road,POINT (103.93056 1.32474)


In [58]:
# find closest hawker centre for each resale flat 
closest_hc = nearest_neighbor(gdf_resale, hawker_centre, return_dist=True)

# Check results 
closest_hc

,ADDRESSSTR,geometry,distance
0,Ang Mo Kio Ave 4,POINT (103.83921 1.37449),178.460502
1,Ang Mo Kio Ave 4,POINT (103.83921 1.37449),178.460502
2,Ang Mo Kio Ave 4,POINT (103.83921 1.37449),178.460502
3,Ang Mo Kio Ave 4,POINT (103.83921 1.37449),178.460502
4,Ang Mo Kio Ave 4,POINT (103.83921 1.37449),178.460502
...,...,...,...
86675,Marine Terrace,POINT (103.91576 1.30578),195.156964
86676,Marine Terrace,POINT (103.91576 1.30578),195.156964
86677,Bendemeer Road,POINT (103.86302 1.31922),493.070279
86678,Clementi West Street 2,POINT (103.76426 1.30378),54.536296


In [59]:
# Rename the geometry of closest stops gdf so that we can easily identify it
closest_hc = closest_hc.rename(columns={'geometry': 'closest_hc_geom'})
closest_hc = closest_hc.rename(columns={'distance': 'distance_hawker_centre'})
closest_hc = closest_hc [['distance_hawker_centre']]

In [60]:
closest_hc

,distance_hawker_centre
0,178.460502
1,178.460502
2,178.460502
3,178.460502
4,178.460502
...,...
86675,195.156964
86676,195.156964
86677,493.070279
86678,54.536296


In [61]:
# join to existing resale dataset 
resale = resale.join(closest_hc)
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,longitude,geometry,closest_site_geom,distance,cbd_distance,distance_mrt,distance_bus_stop,distance_mall,distance_school,distance_hawker_centre
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,...,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,...,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,...,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,...,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,...,103.837616,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,...,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420,315.359851,195.156964
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,...,103.914040,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420,315.359851,195.156964
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,...,103.865166,POINT (103.86517 1.33541),POINT (103.86555 1.33062),134.881667,6.028659,444.669074,15.509091,306.029520,107.350259,493.070279
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,...,103.763817,POINT (103.76382 1.30462),POINT (103.77302 1.34057),1397.163127,10.018078,320.164404,78.858626,254.161399,1363.610294,54.536296


### Distance to Nearest Supermarket 

In [62]:
# read in location data 
supermarkets = gpd.read_file('supermarkets/SUPERMARKETS.shp')
# set crs
supermarkets.crs = {'init' :'epsg:3414'}

//anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [63]:
# extract only name and geometry columns
supermarkets = supermarkets[['LIC_NAME','geometry']]
supermarkets

,LIC_NAME,geometry
0,LI LI CHENG SUPERMARKET (PUNGGOL) PTE. LTD.,POINT (35561.223 42685.167)
1,SHENG SIONG SUPERMARKET PTE LTD,POINT (32184.014 32947.461)
2,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (33903.476 39480.464)
3,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (37083.820 35017.471)
4,YES SUPERMARKET PTE LTD,POINT (41320.297 37283.818)
...,...,...
521,I-TEC SUPERMART PTE LTD,POINT (33768.694 32842.360)
522,YEO HOCK HENG,POINT (33404.177 32874.136)
523,AZ SUPERMART PTE. LTD.,POINT (36527.891 35585.588)
524,FOODIE MARKET PLACE PTE. LTD.,POINT (34992.518 32046.000)


In [64]:
# reproject data to WGS84 
supermarkets = supermarkets.to_crs(epsg=4326)
supermarkets

,LIC_NAME,geometry
0,LI LI CHENG SUPERMARKET (PUNGGOL) PTE. LTD.,POINT (103.90126 1.40230)
1,SHENG SIONG SUPERMARKET PTE LTD,POINT (103.87091 1.31424)
2,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.88637 1.37332)
3,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.91494 1.33296)
4,YES SUPERMARKET PTE LTD,POINT (103.95301 1.35345)
...,...,...
521,I-TEC SUPERMART PTE LTD,POINT (103.88515 1.31329)
522,YEO HOCK HENG,POINT (103.88188 1.31358)
523,AZ SUPERMART PTE. LTD.,POINT (103.90995 1.33810)
524,FOODIE MARKET PLACE PTE. LTD.,POINT (103.89615 1.30609)


In [65]:
# find closest supermarket for each resale flat 
closest_supermarket = nearest_neighbor(gdf_resale,supermarkets,return_dist=True)
# check results 
closest_supermarket

,LIC_NAME,geometry,distance
0,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.83949 1.37424),209.539767
1,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.83949 1.37424),209.539767
2,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.83949 1.37424),209.539767
3,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.83949 1.37424),209.539767
4,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.83949 1.37424),209.539767
...,...,...,...
86675,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.91538 1.30561),152.227032
86676,COLD STORAGE SINGAPORE (1983) PTE LTD,POINT (103.91538 1.30561),152.227032
86677,HAO MART PTE. LTD.,POINT (103.86339 1.33545),197.676735
86678,SHENG SIONG SUPERMARKET PTE LTD,POINT (103.76375 1.30246),57.632709


In [66]:
# Rename the geometry of closest stops gdf so that we can easily identify it
closest_supermarket = closest_supermarket.rename(columns={'geometry': 'closest_supermarket_geom'})
closest_supermarket = closest_supermarket.rename(columns={'distance': 'distance_supermarket'})
closest_supermarket = closest_supermarket[['distance_supermarket']]

In [67]:
# join to resale dataset 

resale = resale.join(closest_supermarket)
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,geometry,closest_site_geom,distance,cbd_distance,distance_mrt,distance_bus_stop,distance_mall,distance_school,distance_hawker_centre,distance_supermarket
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,...,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,...,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,...,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,...,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,...,POINT (103.83762 1.37510),POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,...,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420,315.359851,195.156964,152.227032
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,...,POINT (103.91404 1.30437),POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420,315.359851,195.156964,152.227032
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,...,POINT (103.86517 1.33541),POINT (103.86555 1.33062),134.881667,6.028659,444.669074,15.509091,306.029520,107.350259,493.070279,197.676735
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,...,POINT (103.76382 1.30462),POINT (103.77302 1.34057),1397.163127,10.018078,320.164404,78.858626,254.161399,1363.610294,54.536296,57.632709


### Distance to Nearest Carpark 

In [68]:
# read in location data 
hdb_carparks = pd.read_csv('hdb-carpark/hdb-carpark-information.csv')
hdb_carparks

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.80,Y
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.10,N
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.00,N
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.00,N
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.00,N
...,...,...,...,...,...,...,...,...,...,...,...,...
2130,Y77L,"BLKS 462A, 462B YISHUN AVENUE 6",29832.1087,45462.7555,SURFACE CAR PARK,ELECTRONIC PARKING,NO,NO,NO,0,4.50,N
2131,Y77M,BLK 461 YISHUN AVENUE 6,29850.1522,45576.0125,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,10,2.15,N
2132,Y78M,BLK 468 YISHUN ST 43,30057.2209,45166.4820,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,4,2.15,N
2133,Y8,"BLK 731/746 YISHUN STREET 71,72/AVENUE 5",27772.9219,45686.2734,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,4.50,N


In [69]:
# create geometry column 
geometry = [Point(xy) for xy in zip(hdb_carparks['x_coord'], hdb_carparks['y_coord'])]
# Coordinate reference system : WGS84
# Creating a Geographic data frame 
hdb_carparks = gpd.GeoDataFrame(hdb_carparks, geometry=geometry)
hdb_carparks.crs = {'init' :'epsg:3414'}
hdb_carparks=hdb_carparks.to_crs({'init' :'epsg:4326'})
hdb_carparks

//anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement,geometry
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,1,1.80,Y,POINT (103.85412 1.30106)
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,5,2.10,N,POINT (103.88506 1.32100)
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,0.00,N,POINT (103.84462 1.32828)
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,0,0.00,N,POINT (103.83498 1.36909)
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,0,0.00,N,POINT (103.84664 1.36612)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2130,Y77L,"BLKS 462A, 462B YISHUN AVENUE 6",29832.1087,45462.7555,SURFACE CAR PARK,ELECTRONIC PARKING,NO,NO,NO,0,4.50,N,POINT (103.84978 1.42742)
2131,Y77M,BLK 461 YISHUN AVENUE 6,29850.1522,45576.0125,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,10,2.15,N,POINT (103.84994 1.42845)
2132,Y78M,BLK 468 YISHUN ST 43,30057.2209,45166.4820,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,4,2.15,N,POINT (103.85180 1.42474)
2133,Y8,"BLK 731/746 YISHUN STREET 71,72/AVENUE 5",27772.9219,45686.2734,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,4.50,N,POINT (103.83128 1.42944)


In [70]:
# find closest carpark for each resale flat 
closest_carpark = nearest_neighbor(gdf_resale,hdb_carparks,return_dist=True)
# check results 
closest_carpark

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,car_park_decks,gantry_height,car_park_basement,geometry,distance
0,A99,BLK 181 ANG MO KIO AVE 5,28433.4999,39885.5496,SURFACE CAR PARK,COUPON PARKING,7AM-10.30PM,NO,NO,0,0.00,N,POINT (103.83721 1.37699),67.225111
1,A99,BLK 181 ANG MO KIO AVE 5,28433.4999,39885.5496,SURFACE CAR PARK,COUPON PARKING,7AM-10.30PM,NO,NO,0,0.00,N,POINT (103.83721 1.37699),67.225111
2,A99,BLK 181 ANG MO KIO AVE 5,28433.4999,39885.5496,SURFACE CAR PARK,COUPON PARKING,7AM-10.30PM,NO,NO,0,0.00,N,POINT (103.83721 1.37699),67.225111
3,A99,BLK 181 ANG MO KIO AVE 5,28433.4999,39885.5496,SURFACE CAR PARK,COUPON PARKING,7AM-10.30PM,NO,NO,0,0.00,N,POINT (103.83721 1.37699),67.225111
4,A99,BLK 181 ANG MO KIO AVE 5,28433.4999,39885.5496,SURFACE CAR PARK,COUPON PARKING,7AM-10.30PM,NO,NO,0,0.00,N,POINT (103.83721 1.37699),67.225111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,MP4M,BLK 16A MARINE TERRACE,37001.0531,31814.9830,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,4,2.05,N,POINT (103.91420 1.30400),20.191316
86676,MP4M,BLK 16A MARINE TERRACE,37001.0531,31814.9830,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,4,2.05,N,POINT (103.91420 1.30400),20.191316
86677,PP3,BLK 134 TO 142 POTONG PASIR AVENUE 3,31619.7981,35088.4721,SURFACE CAR PARK,ELECTRONIC PARKING,NO,SUN & PH FR 7AM-10.30PM,YES,0,0.00,N,POINT (103.86584 1.33360),89.616541
86678,C37,"BLK 720-727,730-731 CLEMENTI WEST STREET 2",20282.3230,31764.2687,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,0,2.15,N,POINT (103.76397 1.30354),33.509746


In [71]:
# Rename the geometry of closest stops gdf so that we can easily identify it
closest_carpark = closest_carpark.rename(columns={'geometry': 'closest_carpark_geom'})
closest_carpark = closest_carpark.rename(columns={'distance': 'distance_carpark'})
closest_carpark = closest_carpark[['distance_carpark']]
closest_carpark

,distance_carpark
0,67.225111
1,67.225111
2,67.225111
3,67.225111
4,67.225111
...,...
86675,20.191316
86676,20.191316
86677,89.616541
86678,33.509746


In [72]:
# join to dataset 

resale = resale.join(closest_carpark)
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,closest_site_geom,distance,cbd_distance,distance_mrt,distance_bus_stop,distance_mall,distance_school,distance_hawker_centre,distance_supermarket,distance_carpark
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,...,POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,...,POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,...,POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,...,POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,...,POINT (103.82861 1.36511),1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,...,POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420,315.359851,195.156964,152.227032,20.191316
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,...,POINT (103.91261 1.30931),206.692805,7.368307,462.438213,96.899174,977.791420,315.359851,195.156964,152.227032,20.191316
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,...,POINT (103.86555 1.33062),134.881667,6.028659,444.669074,15.509091,306.029520,107.350259,493.070279,197.676735,89.616541
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,...,POINT (103.77302 1.34057),1397.163127,10.018078,320.164404,78.858626,254.161399,1363.610294,54.536296,57.632709,33.509746


### Distance to Nearest Park/Green Space 

In [73]:
# read in location data 
parks = gpd.read_file('nationalparks/NATIONALPARKS.shp')
parks
# set crs 
parks.crs = {'init' :'epsg:3414'}


//anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [74]:
# extract relevant columns 
parks = parks[['NAME','geometry']]
# reproject data 
parks = parks.to_crs(epsg = 4326)
parks

,NAME,geometry
0,Jalan Pelatok Open Space,POINT (103.96086 1.34618)
1,Turnhouse Park,POINT (103.97898 1.38990)
2,Coney Island Park,POINT (103.92313 1.40991)
3,Stratton Walk Playground,POINT (103.86697 1.37924)
4,Leban Park,POINT (103.82907 1.37276)
...,...,...
345,Realty Park,POINT (103.89036 1.36796)
346,Changi Point Promenade,POINT (103.98919 1.39017)
347,Aroozoo Avenue Playground,POINT (103.88595 1.35520)
348,St. Anne's Wood Playground,POINT (103.90374 1.39164)


In [75]:
# find nearest park 
closest_park = nearest_neighbor(gdf_resale, parks, return_dist=True)
# rename columns
closest_park = closest_park.rename(columns={'geometry': 'closest_park_geom'})
closest_park = closest_park.rename(columns={'distance': 'distance_park'})
# check results
closest_park

,NAME,closest_park_geom,distance_park
0,Yio Chu Kang Gardens Playground,POINT (103.83749 1.38068),149.181000
1,Yio Chu Kang Gardens Playground,POINT (103.83749 1.38068),149.181000
2,Yio Chu Kang Gardens Playground,POINT (103.83749 1.38068),149.181000
3,Yio Chu Kang Gardens Playground,POINT (103.83749 1.38068),149.181000
4,Yio Chu Kang Gardens Playground,POINT (103.83749 1.38068),149.181000
...,...,...,...
86675,Telok Kurau Lorong N Park,POINT (103.91281 1.31032),210.194891
86676,Telok Kurau Lorong N Park,POINT (103.91281 1.31032),210.194891
86677,Carmichael Road Playground,POINT (103.86311 1.34262),298.115635
86678,Jalan Mas Kuning Playground,POINT (103.76361 1.30781),87.412127


In [76]:
closest_park = closest_park[['distance_park']]
# join data 
resale = resale.join(closest_park)
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,distance,cbd_distance,distance_mrt,distance_bus_stop,distance_mall,distance_school,distance_hawker_centre,distance_supermarket,distance_carpark,distance_park
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,...,1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111,149.181000
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,...,1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111,149.181000
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,...,1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111,149.181000
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,...,1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111,149.181000
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,...,1036.226326,10.353154,730.029000,23.365871,933.145397,209.449588,178.460502,209.539767,67.225111,149.181000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,...,206.692805,7.368307,462.438213,96.899174,977.791420,315.359851,195.156964,152.227032,20.191316,210.194891
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,...,206.692805,7.368307,462.438213,96.899174,977.791420,315.359851,195.156964,152.227032,20.191316,210.194891
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,...,134.881667,6.028659,444.669074,15.509091,306.029520,107.350259,493.070279,197.676735,89.616541,298.115635
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,...,1397.163127,10.018078,320.164404,78.858626,254.161399,1363.610294,54.536296,57.632709,33.509746,87.412127


# Feature Engineering II - Grouping Flats by Location  

In [ ]:
resale

In [77]:
# Grouping observations by distance to heritage site 
# Creating dummy variables 


resale['distance'].describe()

# min value as 22m and max as 1784m 
# therefore, we create categories # 0-400, 400-800,800-1200,1200-1600,1600-2000m,Beyond 2000m etc 


count    86680.000000
mean       582.870933
std        466.396914
min         20.155053
25%        212.623382
50%        346.603610
75%       1008.921656
max       1784.150485
Name: distance, dtype: float64

In [78]:
# 0- 400m dummy
test_df = resale[['distance']]
test_df['0-400m'] = (test_df.distance <= 400.).astype('int')
test_df

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,distance,0-400m
0,1036.226326,0
1,1036.226326,0
2,1036.226326,0
3,1036.226326,0
4,1036.226326,0
...,...,...
86675,206.692805,1
86676,206.692805,1
86677,134.881667,1
86678,1397.163127,0


In [79]:
# 400-800m
bool_series = test_df['distance'].between(400, 800, inclusive = False) 
test_df['400-800m']=bool_series.astype('int')
test_df

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,distance,0-400m,400-800m
0,1036.226326,0,0
1,1036.226326,0,0
2,1036.226326,0,0
3,1036.226326,0,0
4,1036.226326,0,0
...,...,...,...
86675,206.692805,1,0
86676,206.692805,1,0
86677,134.881667,1,0
86678,1397.163127,0,0


In [80]:
# 800-1200m
bool_series = test_df['distance'].between(800, 1200, inclusive = False) 
test_df['800-1200m']=bool_series.astype('int')
test_df

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,distance,0-400m,400-800m,800-1200m
0,1036.226326,0,0,1
1,1036.226326,0,0,1
2,1036.226326,0,0,1
3,1036.226326,0,0,1
4,1036.226326,0,0,1
...,...,...,...,...
86675,206.692805,1,0,0
86676,206.692805,1,0,0
86677,134.881667,1,0,0
86678,1397.163127,0,0,0


In [81]:
# 1200-1600m
bool_series = test_df['distance'].between(1200, 1600, inclusive = False) 
test_df['1200-1600m']=bool_series.astype('int')
test_df

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,distance,0-400m,400-800m,800-1200m,1200-1600m
0,1036.226326,0,0,1,0
1,1036.226326,0,0,1,0
2,1036.226326,0,0,1,0
3,1036.226326,0,0,1,0
4,1036.226326,0,0,1,0
...,...,...,...,...,...
86675,206.692805,1,0,0,0
86676,206.692805,1,0,0,0
86677,134.881667,1,0,0,0
86678,1397.163127,0,0,0,1


In [82]:
# 1600-2000m
bool_series = test_df['distance'].between(1600, 2000, inclusive = False) 
test_df['1600-2000m']=bool_series.astype('int')
test_df

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,distance,0-400m,400-800m,800-1200m,1200-1600m,1600-2000m
0,1036.226326,0,0,1,0,0
1,1036.226326,0,0,1,0,0
2,1036.226326,0,0,1,0,0
3,1036.226326,0,0,1,0,0
4,1036.226326,0,0,1,0,0
...,...,...,...,...,...,...
86675,206.692805,1,0,0,0,0
86676,206.692805,1,0,0,0,0
86677,134.881667,1,0,0,0,0
86678,1397.163127,0,0,0,1,0


In [83]:
test_df= test_df.drop(['distance'], axis = 1)


In [84]:
resale = resale.join(test_df)

In [85]:
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,distance_school,distance_hawker_centre,distance_supermarket,distance_carpark,distance_park,0-400m,400-800m,800-1200m,1200-1600m,1600-2000m
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,...,209.449588,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,...,209.449588,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,...,209.449588,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,...,209.449588,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,...,209.449588,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,...,315.359851,195.156964,152.227032,20.191316,210.194891,1,0,0,0,0
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,...,315.359851,195.156964,152.227032,20.191316,210.194891,1,0,0,0,0
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,...,107.350259,493.070279,197.676735,89.616541,298.115635,1,0,0,0,0
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,...,1363.610294,54.536296,57.632709,33.509746,87.412127,0,0,0,1,0


# Data Cleaning and Pre-Processing 

### Floor Level

In [86]:
# floor level 
resale['storey_range'].unique()


array(['07 TO 09', '10 TO 12', '04 TO 06', '01 TO 03', '13 TO 15',
       '22 TO 24', '16 TO 18', '25 TO 27', '19 TO 21', '28 TO 30',
       '37 TO 39', '31 TO 33', '34 TO 36', '40 TO 42'], dtype=object)

In [87]:
# define function to process storey range data 
def get_mean_floor(x):
    return int((int(x[:2])+ int (x[-2:]))*0.5)

In [88]:
# add processed storey column 
resale['storey']= resale['storey_range'].map(lambda x: get_mean_floor(x))

In [89]:
resale

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,distance_hawker_centre,distance_supermarket,distance_carpark,distance_park,0-400m,400-800m,800-1200m,1200-1600m,1600-2000m,storey
0,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,...,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0,8
1,2015,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,69,...,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0,11
2,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,68,...,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0,5
3,2016,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,68,...,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0,8
4,2016,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,68,...,178.460502,209.539767,67.225111,149.181000,0,0,1,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,43.0,Improved,1975,63,...,195.156964,152.227032,20.191316,210.194891,1,0,0,0,0,8
86676,2011,MARINE PARADE,2 ROOM,16,MARINE TER,07 TO 09,42.0,Improved,1975,63,...,195.156964,152.227032,20.191316,210.194891,1,0,0,0,0,8
86677,2011,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,72,...,493.070279,197.676735,89.616541,298.115635,1,0,0,0,0,2
86678,2011,CLEMENTI,3 ROOM,727,CLEMENTI WEST ST 2,01 TO 03,84.0,New Generation,1985,73,...,54.536296,57.632709,33.509746,87.412127,0,0,0,1,0,2


### Flat Model, Type and Town

In [90]:
# ONE-HOT ENCODING 

y = resale[['flat_type','flat_model','town']]
y = pd.get_dummies(y)


In [91]:
y

,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_model_Adjoined flat,flat_model_Apartment,flat_model_DBSS,flat_model_Improved,...,flat_model_Standard,flat_model_Terrace,town_ANG MO KIO,town_BUKIT MERAH,town_BUKIT TIMAH,town_CLEMENTI,town_GEYLANG,town_MARINE PARADE,town_QUEENSTOWN,town_TOA PAYOH
0,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86675,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
86676,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
86677,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
86678,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [92]:
resale = resale.join(y)

In [93]:
resale.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 86680 entries, 0 to 86679
Data columns (total 58 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   month                          86680 non-null  int64   
 1   town                           86680 non-null  object  
 2   flat_type                      86680 non-null  object  
 3   block                          86680 non-null  object  
 4   street_name                    86680 non-null  object  
 5   storey_range                   86680 non-null  object  
 6   floor_area_sqm                 86680 non-null  float64 
 7   flat_model                     86680 non-null  object  
 8   lease_commence_date            86680 non-null  int64   
 9   remaining_lease                86680 non-null  int64   
 10  resale_price                   86680 non-null  float64 
 11  address                        86680 non-null  object  
 12  latitude                

In [94]:
# drop unnecessary columns 

test = resale.drop(['geometry','block','street_name','lease_commence_date','storey_range','closest_site_geom','flat_type','flat_model','town'], axis=1)
test.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 86680 entries, 0 to 86679
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   month                          86680 non-null  int64  
 1   floor_area_sqm                 86680 non-null  float64
 2   remaining_lease                86680 non-null  int64  
 3   resale_price                   86680 non-null  float64
 4   address                        86680 non-null  object 
 5   latitude                       86680 non-null  float32
 6   longitude                      86680 non-null  float32
 7   distance                       86680 non-null  float64
 8   cbd_distance                   86680 non-null  float64
 9   distance_mrt                   86680 non-null  float64
 10  distance_bus_stop              86680 non-null  float64
 11  distance_mall                  86680 non-null  float64
 12  distance_school                86680 n

# Exporting Data for Further Analysis in R 

In [95]:
# Export final dataframe as csv for further analysis in R 
test = pd.DataFrame(test)
test.to_csv('FINAL_export_resale_flats.csv')